In [1]:
!pip install --quiet py3Dmol

In [11]:
%%shell
URL=https://github.com/HanaJaafari/Exon_Frustration_Analysis/raw/main/exon_ali_data.zip
FILE="${URL##*/}"

#if [ -e "sample_data" ]; then rm -r sample_data; fi
if [ ! -e $FILE ]; then wget --quiet $URL; unzip -qq $FILE; fi

In [12]:
import numpy as np
import pandas as pd

import pickle
import scipy.signal as scs

import matplotlib.pyplot as plt
import seaborn as sns

import py3Dmol
from matplotlib import pyplot as plt, colors
from matplotlib.colors import Normalize
from matplotlib import cm
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator
import matplotlib
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [14]:
path_='exon_ali_data/'
results_table_pdb=pd.read_csv(path_+'results_table_pdb.csv',index_col=0)
results_table_pdb=results_table_pdb.drop_duplicates(subset=['pfam'],ignore_index=True)

In [24]:
def map_hist_to_colors(h_,vmin,vmax):
    cmap=cm.get_cmap('cool')
    norm = Normalize(vmin,vmax)
    rgba_values = cmap(norm(h_))
    colors=[]
    for rgba in rgba_values:
        colors.append(matplotlib.colors.rgb2hex(rgba))  
    return colors

def view_3d_exon_hist(ali_seq,colors,pdb,AF=False):

    if AF:
        pdb_filename='/home/ezequiel/SynologyDrive/folding_ank/pdbs/'+pdb+'.pdb'

        view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
        view.addModel(open(pdb_filename,'r').read(),'pdb')
        
    else:
        view = py3Dmol.view(query='pdb:'+pdb,width=800, height=600)
        
    view.setBackgroundColor('white')

    view.setStyle({'cartoon':{'color':'white'}})
    view.setStyle({'chain':'B'},{'opacity':0 })
    view.setStyle({'chain':'C'},{'opacity':0 })
    view.setStyle({'chain':'D'},{'opacity':0 })
    view.setStyle({'chain':'E'},{'opacity':0 })
    view.setStyle({'chain':'F'},{'opacity':0 })
    view.setStyle({'chain':'G'},{'opacity':0 })

    if len(ali_seq.shape)>1: #choose the instance with less gaps
        count_aa=[]
        for seqs in ali_seq:
            count_aa.append(len(seqs[seqs>0]))
        print(np.argmax(count_aa))
        ali_seq=ali_seq[np.argmax(count_aa)]
    
    #change residue color
    for i,res in enumerate(ali_seq):
        if res>0: #gaps are res=-1 and we dont want them
            view.addStyle({'chain':'A','resi':[str(res)]},{'cartoon':{'color':colors[i]}})
        #view.addStyle({'chain':'A','resi':[str(alinum.no[e_])]},{'cartoon':{'color':'white'}})
    view.zoomTo(viewer=(0,0))
    return view

    # esto no toma en cuenta si hay varias columnas juntas chicas (problemas de alineamiento)
def find_common_bs(exon_freq,order,thresh,border,npos):
    final_bs=scs.argrelmax(exon_freq,order=order)[0]
    final_bs=final_bs[exon_freq[final_bs]>thresh]
    final_bs=final_bs[final_bs>=border]
    final_bs=final_bs[final_bs<=(npos-border)]
    final_bs=np.hstack([0,final_bs,npos])
    return final_bs

# plot exon histograms and view pdb

In [25]:
colors= sns.color_palette().as_hex() + list(pd.read_csv(path_+'colors.csv')['0'].values)


In [26]:
@interact
def plot_(family=list(results_table_pdb.name), 
          order=widgets.IntSlider(min=1, max=50, step=1, value=10), 
          threshold=widgets.FloatSlider(min=0.00, max=0.025, step=0.005, value=0.01,readout_format='.3f'), 
          no_insert_junctions=False,
          filter_=['both_ex_sym','any_exon_sym','intron_ph','kingdom','exon_bs_ins'],
          class_=['all',True,False,0,1,2],
          kingdom=['all','Fungi', 'Heunggongvirae', 'Metazoa', 'Viridiplantae', 'unknown'],
          multiple=['stack','layer', 'dodge','fill'],
          panch_foldons=[False,True],
          pdb_view=['common exons','boundaries','no']):
    border=order
    f=family
    path_f=path_+f+'/'
    with open (path_+'exon_pdb_ali/exon_pdb_ali_'+f, 'rb') as fp:
        exon_freq_old, ali_seq_num_pdb, pdb = pickle.load(fp)
            
    exon_table=pd.read_csv(path_f+'exon_table.csv')
    
    if no_insert_junctions:
        exon_table=exon_table[~exon_table.exon_bs_ins]
    
    if kingdom!='all':
        exon_table=exon_table[exon_table.kingdom==kingdom]
        if len(exon_table[exon_table.kingdom==kingdom])==0:
            print('no organisms for',kingdom)
            return 0
        
    npos=len(exon_freq_old)        
    fig1,ax1=plt.subplots(figsize=(18,3.5),facecolor='white')
    
    if panch_foldons:
        with open (path_+'panchenko_foldons_aligned', 'rb') as fp:
              panchenko_foldons_aligned = pickle.load(fp)
        ib=panchenko_foldons_aligned[panchenko_foldons_aligned.name==f].index[0]
        bs=[item for sublist in panchenko_foldons_aligned.panchenko_foldon_bs[ib] for item in sublist]
        
        if len(bs)==0:
            print('no panchenko foldons for this family')
        else:
            for b in bs:
                ax1.axvline(b,color='blue') 
    
    if pdb_view=='no':
       
    
        h=sns.histplot(ax=ax1,data=exon_table, x="exon_bs_aa",bins=np.arange(-0.5,npos+0.5),
             hue=filter_,multiple=multiple,weights="exon_rew")
        return 0

    else:
        if class_=='all':
            print('Total counts =',int(sum(exon_table.exon_rew)) )
            exon_freq=np.histogram(exon_table.exon_bs_aa,bins=np.arange(-0.5,npos+0.5),density=True,
                                   weights=exon_table.exon_rew)[0]

        elif len(exon_table[exon_table[filter_]==class_].exon_bs_aa)==0:
            print('ERROR: wrong class')
            return 0

        else:
            print('Total counts =',int(sum(exon_table[exon_table[filter_]==class_].exon_rew)) )
            exon_freq=np.histogram(exon_table[exon_table[filter_]==class_].exon_bs_aa,bins=np.arange(-0.5,npos+0.5),
                                   density=True,weights=exon_table[exon_table[filter_]==class_].exon_rew )[0]

        ax1.bar(x=np.arange(npos),height=exon_freq,color=map_hist_to_colors(exon_freq,vmin=0,vmax=max(exon_freq)))

        if pdb_view=='boundaries':
            view=view_3d_exon_hist(ali_seq_num_pdb, colors=map_hist_to_colors(exon_freq,vmin=0,vmax=max(exon_freq)),
                               pdb=pdb)
        if pdb_view=='common exons':
            
            
            final_bs=find_common_bs(exon_freq,order,threshold,border,npos)
            print('common bs =',final_bs)
            ax1.bar(x=np.arange(npos),height=exon_freq,color=map_hist_to_colors(exon_freq,vmin=0,
                                                                                vmax=max(exon_freq)))

            ax1.scatter(x=final_bs[1:-1],y=exon_freq[final_bs[1:-1]]+max(exon_freq[final_bs[1:-1]])*0.05,
                        color='red',s=100,marker='*')

            for i in range(0,len(final_bs)-1):
                ax1.hlines(y=max(exon_freq)*1.2, xmin=final_bs[i], 
                           xmax=final_bs[i+1], linewidth=2, color=colors[i],lw=10)
 
     
            ax1.hlines(y=threshold, xmin=0, xmax=npos, color='grey',lw=0.5,ls='--')
            ax1.vlines(x=border, ymin=0, ymax=max(exon_freq)*1.2, color='grey',lw=0.5,ls='--')
            ax1.vlines(x=npos-border, ymin=0, ymax=max(exon_freq)*1.2, color='grey',lw=0.5,ls='--')

            
            view=view_3d_exon_hist(ali_seq_num_pdb,
                  np.hstack([np.repeat(colors[i],final_bs[i+1]-final_bs[i]) for i in range(len(final_bs)-1)]),
                  pdb)

        return view
    

interactive(children=(Dropdown(description='family', options=('Ribonuclease', 'Cytochrom_C', 'Lys', 'Globin', …